# Master Thesis on the Semantics of (made-up) Names

* Author: Aron Joosse
* Supervisor: Giovanni Cassani
* Institution: Tilburg University

Can take inspiration from: https://github.com/Masetto96/BA-Thesis-form-meaning-mapping/blob/master/form_meaning_mapping.ipynb

# Library Imports

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 1.4 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.9.1-py2.py3-none-any.whl (211 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-linux_x86_64.whl size=304211 sha256=58d1c0bdc3e06d11092411b2e43ea2414be15a74b4fb0bba1926b67cf3570aa6
  Stored in directory: /home/jovyan/.cache/pip/wheels/64/57/bc/1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext


In [ ]:
import fasttext
import nltk


# Data Import

## COCA

In [ ]:
AA

## Names

# Preprocessing

## Cleaning Corpus

In [ ]:
 def clean_corpus(data):
    # Separate sentences
    
    # Split sentences into wordlists
    
    # 
    
    # DO NOT REMOVE PRONOUNS, HIM HER ETC.
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    stop_words.remove('him')
    stop_words.remove('her')
    stop_words.remove('hers')
    stop_words.remove('his')
    stop_words.remove('he')
    stop_words.remove('she')
    no_stop = []
    for sent in lemma_sent:
        tokens_without_sw = [word for word in sent if not word in stop_words]
        no_stop.append(tokens_without_sw)

## Training fastText and Validating on Word Embeddings Benchmark

In [ ]:
# Skipgram model :
#model = fasttext.train_unsupervised('data.txt', model='skipgram')

#model.save_model("model_filename.bin")

#model = fasttext.load_model("model_filename.bin")

#model.get_nearest_neighbors('asparagus')

#In a similar spirit, one can play around with word analogies. For example, we can see if our model can guess what is to France, and what Berlin is to Germany.
#This can be done with the analogies functionality. It takes a word triplet (like Germany Berlin France) and outputs the analogy:
#model.get_analogies("berlin", "germany", "france")